In [1]:
import numpy as np
import pandas as pd

Merge Environmental Data

In [ ]:
# load in and merge seasonal data
# temp, humidity, pr
temp_seasonal = pd.read_csv("../../ci3_confounders/data_for_analysis/earth_engine/temperature/temperature_seasonal_zipcode_combined.csv")

# wind
# wind_seasonal = pd.read_csv("../../ci3_confounders/data_for_analysis/earth_engine/wind/wind_seasonal_zipcode_combined.csv")

# pm, ozone, no2
pm_seasonal = pd.read_csv("../../ci3_exposure/pm25/whole_us/seasonal/zipcode/pm25_seasonalavg_zipcode.csv")

ozone_seasonal = pd.read_csv("../../ci3_exposure/ozone/whole_us/seasonal/zipcode/ozone_seasonalavg_zipcode.csv")

no2_seasonal = pd.read_csv("../../ci3_exposure/no2/whole_us/seasonal/zipcode/no2_seasonalavg_zipcode.csv")

In [ ]:
# drop nan values
dfs = [temp_seasonal,pm_seasonal,ozone_seasonal,no2_seasonal]
for df in dfs:
    df.dropna(inplace=True)

In [ ]:
# merge together on ZIP and year
merged_df1 = pm_seasonal.merge(ozone_seasonal,how='left',left_on=['ZIP','year'],right_on=['ZIP','year'])
merged_df2 = merged_df1.merge(no2_seasonal,how='left',left_on=['ZIP','year'],right_on=['ZIP','year'])
merged_df3 = merged_df2.merge(temp_seasonal,how='left',left_on=['ZIP','year'],right_on=['ZIP','year'])

# drop nan and save
seasonal_data = merged_df3.dropna()
seasonal_data.to_csv("seasonal_environmental_data.csv")

Calculate Moving Averages

In [2]:
s_df = pd.read_csv("seasonal_environmental_data.csv")

In [81]:
def calc_rolling_avg(df, cols_to_avg, avg_num):
        
    for c in cols_to_avg:
        # calculate 4 year average
        avg = (df.assign(a = df.rolling(window=avg_num)[c].mean()).groupby([df['ZIP'],df['year']])['a'].last())
        # assign to df
        df[c + '_' + str(avg_num) + 'y_avg'] = list(avg.values)
        
    return df

In [82]:
cols = ['pm25_summer', 'pm25_winter', 'pm25_fall',
   'pm25_spring', 'ozone_summer', 'ozone_winter', 'ozone_fall',
   'ozone_spring', 'no2_summer', 'no2_winter', 'no2_fall', 'no2_spring',
   'summer_tmmx', 'summer_rmax', 'winter_tmmx', 'winter_rmax']

s_df = s_df[['ZIP','year']+cols]
# df_w_avg = calc_rolling_avg(s_df, cols, 4)

In [83]:
# avg = s_df.groupby(['ZIP','year']).assign(a = s_df.rolling(window=4)['pm25_summer'].mean()) #.groupby([s_df['ZIP','year']])['a'].last())
# avg.ozone_spring_4y_avg.
import warnings
warnings.filterwarnings('ignore')

cols = ['pm25_summer', 'pm25_winter', 'pm25_fall',
   'pm25_spring', 'ozone_summer', 'ozone_winter', 'ozone_fall',
   'ozone_spring', 'no2_summer', 'no2_winter', 'no2_fall', 'no2_spring',
   'summer_tmmx', 'summer_rmax', 'winter_tmmx', 'winter_rmax']

zips = np.unique(s_df.ZIP)
df_w_avg_vals = []

for i,z in enumerate(zips):
    df_sub = s_df[s_df.ZIP == z]
    for c in cols:
        avg = (df_sub.assign(a = df_sub.rolling(window=4)[c].mean()).groupby([df_sub['year']])['a'].last())
        df_sub[c + '_' + str(4) + 'y_avg'] = list(avg.values)
        
    df_sub2 = df_sub.drop(columns = cols)
    df_w_avg_vals.append(df_sub2)

df_w_avg = pd.concat(df_w_avg_vals)

Merge for 2011-2016

In [96]:
# now merge with medicare data
med_data = pd.read_csv("medicare_hosp_merged_zip_2011_2016.csv").drop(columns=['Unnamed: 0'])

In [98]:
merged_data_full_avg = med_data.merge(df_w_avg,how='left',left_on=['zip','year'],right_on=['ZIP','year'])
merged_data_full_avg2 = merged_data_full_avg.dropna()